# 🎓 UQ Course Scraper & Data Engineer
**Project:** UQ Course Navigator & Grade Tracker  
**Phase:** 1 - Data Engineering  
**Status:** ✅ Ready for Phase 2 (Database)

---
**Mục tiêu:**
1. Thu thập thông tin chi tiết các môn học (Description, Units, Contact Hours).
2. Truy cập Electronic Course Profile (ECP) để lấy bảng điểm (Assessments).
3. Xử lý dữ liệu thô: Gắn cờ (Hurdle, Team-based), tính trọng số %.

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re
import pandas as pd # Dùng pandas để xem bảng cho đẹp trong Notebook
from tqdm import tqdm # <--- THÊM DÒNG NÀY
import time # <--- THÊM DÒNG NÀY (để dùng sleep tránh bị chặn IP)
import concurrent.futures
import os


## 🛠️ 1. Core Scraper Functions
Phần này định nghĩa các hàm xử lý chính:
* **`scrape_uq_course`**: Lấy thông tin tổng quan từ trang chủ môn học.
* **`scrape_assessment_table`**: Đi sâu vào link ECP để bóc tách bảng điểm.
* **`clean_assessment_task`**: Dùng Regex để làm sạch tên bài tập và gắn cờ (`is_hurdle`, `is_in_person`).
* 

In [2]:
def extract_course_codes(text):
    return re.findall(r'[A-Z]{4}\d{4}', text)

def scrape_uq_course(course_code):
    url = f"https://my.uq.edu.au/programs-courses/course.html?course_code={course_code}"
    headers = {"User-Agent": "Mozilla/5.0"}
    
    try:
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            return None
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Helper function để lấy text an toàn
        def get_text(selector_id):
            element = soup.find(id=selector_id)
            return element.get_text(strip=True) if element else "N/A"

        # 1. Thông tin định danh
        full_title = get_text('course-title')
        # Tách lấy tên môn (bỏ phần mã môn trong ngoặc)
        course_name = re.sub(r'\s\([A-Z]{4}\d{4}\)', '', full_title)

        # 2. Thông tin chi tiết (Summary Panel)
        level = get_text('course-level')
        faculty = get_text('course-faculty')
        school = get_text('course-school')
        units = int(get_text('course-units'))
        duration = get_text('course-duration')
        mode = get_text('course-mode')
        contact_hours = soup.find(id='course-contact').get_text(separator=' ', strip=True) if soup.find(id='course-contact') else "N/A"        
        
        # 3. Điều kiện và Ràng buộc
        prereq_raw = get_text('course-prerequisite')
        incomp_raw = get_text('course-incompatible')
        
        # 4. Tóm tắt nội dung & Đánh giá sơ bộ
        description = get_text('course-summary')
        assessment_summary = get_text('course-assessment-methods')
        coordinator = get_text('course-coordinator')

        # 5. Link quan trọng
        ecp_link = ""
        ecp_tag = soup.find('a', class_='profile-available')
        if ecp_tag:
            ecp_link = ecp_tag['href']
            # Nếu link là tương đối, nối thêm domain
            if ecp_link.startswith('/'):
                ecp_link = "https://programs-courses.uq.edu.au" + ecp_link

        return {
            "code": course_code,
            "title": course_name,
            "units": units,
            "level": level,
            "faculty": faculty,
            "school": school,
            "description": description,
            "contact_hours": contact_hours,
            "assessment_summary": assessment_summary,
            "prerequisites_text": prereq_raw,
            "prerequisites_list": extract_course_codes(prereq_raw),
            "incompatible_list": extract_course_codes(incomp_raw),
            "coordinator": coordinator,
            "ecp_link": ecp_link,
            "url": url
        }
    except Exception as e:
        print(f"Error scraping {course_code}: {e}")
        return None

def clean_assessment_task(raw_name):
    # Khởi tạo các flag mặc định là False
    flags = {
        "is_hurdle": False,
        "is_identity_verified": False,
        "is_in_person": False,
        "is_team_based": False
    }
    
    # 1. Kiểm tra sự tồn tại của các từ khóa (không phân biệt hoa thường)
    if re.search(r'hurdle', raw_name, re.IGNORECASE):
        flags["is_hurdle"] = True
    if re.search(r'identity verified', raw_name, re.IGNORECASE):
        flags["is_identity_verified"] = True
    if re.search(r'in-person', raw_name, re.IGNORECASE):
        flags["is_in_person"] = True
    if re.search(r'team', raw_name, re.IGNORECASE):
        flags["is_team_based"] = True
        
    # 2. Xóa các từ khóa này khỏi chuỗi
    # Regex này tìm các từ đó kèm theo dấu phẩy hoặc ngoặc đơn xung quanh chúng
    clean_name = re.sub(r'\(?Hurdle\)?', '', raw_name, flags=re.IGNORECASE)
    clean_name = re.sub(r'\(?Identity Verified\)?', '', clean_name, flags=re.IGNORECASE)
    clean_name = re.sub(r'\(?In-person\)?', '', clean_name, flags=re.IGNORECASE)
    clean_name = re.sub(r'\(?Team or group-based\)?', '', clean_name, flags=re.IGNORECASE)
    
    # 3. Dọn dẹp các ký tự thừa (dấu phẩy dư, khoảng trắng dư)
    clean_name = clean_name.replace(', ,', ',').strip(' ,()')
    clean_name = re.sub(r'\s+', ' ', clean_name) # Xóa khoảng trắng kép
    
    return clean_name, flags

def scrape_assessment_table(ecp_url):
    if not ecp_url or ecp_url == "N/A":
        return []
    
    # Đảm bảo chúng ta vào đúng trang Assessment (Section 5)
    # Lưu ý: Tùy link mà UQ cung cấp, có khi phải append thêm để ra trang full assessment
    # Ở phiên bản V1, ta giả sử link dẫn đến trang có chứa bảng Assessment
    
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        response = requests.get(ecp_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        assessments = []
        
        # Tìm bảng Assessment - UQ thường dùng class 'assessment-details' hoặc tìm theo text 'Assessment Task'
        table = soup.find('section', class_='section section--course-profile section--in-view') 
        
        if not table:
            # Plan B: Tìm table bất kỳ có chứa chữ "Weight"
            tables = soup.find_all('table')
            for t in tables:
                if "Weight" in t.text:
                    table = t
                    break
        
        if table:
            rows = table.find_all('tr')[1:] # Bỏ qua hàng tiêu đề (header)
            for row in rows:
                cols = row.find_all('td')
                if len(cols) >= 2:
                    category = cols[0].get_text(strip=True)
                    assesment_task= cols[1].get_text(strip=True)
                    
                    
                    weight_raw = cols[2].get_text(strip=True)
                    due_date = cols[3].get_text(separator=' ', strip=True) if len(cols) > 3 else "N/A"                    
                    
                    # Clean trọng số: "20%" -> 0.2
                    weight_percent = re.findall(r'\d+', weight_raw)
                    weight_value = int(weight_percent[0]) / 100 if weight_percent else 0
                    
                    # Trong hàm scrape_assessment_table, đoạn xử lý col[0] (task_name):

                    task_name_raw = cols[1].get_text(strip=True)
                    clean_name, flags = clean_assessment_task(task_name_raw)

                    assessments.append({
                        "category": category,
                        "assesment_task": clean_name,
                        "weight": weight_value,
                        "due_date": due_date,
                        "flags": flags
                        
                    })
        
        return assessments
    except Exception as e:
        print(f"Lỗi khi cào bảng điểm tại {ecp_url}: {e}")
        return []

# --- HÀM TỔNG HỢP ---
def get_full_course_data(course_code):
    course_code = course_code.upper()
    # Bước 1: Lấy thông tin chung
    course_data = scrape_uq_course(course_code)
    
    if course_data and course_data['ecp_link']:
        course_data['assessments'] = scrape_assessment_table(course_data['ecp_link'])
        
    return course_data

## 📥 2. Load Input Data
Đọc danh sách mã môn học (Course Codes) từ file `data/eait_codes_only.json` đã được quét ở bước trước (Scan Phase).

In [3]:
import os 
import json

# --- CẤU HÌNH ĐƯỜNG DẪN ---
# Đi từ folder 'scraper' lùi ra ngoài (..) rồi vào folder 'data'
input_path = os.path.join('..', 'data', 'all_course_codes.json')

# Đọc file danh sách mã môn
try:
    with open(input_path, 'r', encoding='utf-8') as f:
        course_list = json.load(f)
    
    print(f"✅ Đã tìm thấy file tại: {input_path}")
    print(f"✅ Đã load thành công {len(course_list)} mã môn.")

except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy file tại '{input_path}'")
    print("👉 Hãy kiểm tra lại xem file json đã nằm trong folder 'data' chưa.")
    course_list = []

✅ Đã tìm thấy file tại: ../data/all_course_codes.json
✅ Đã load thành công 3860 mã môn.


## 🚀 3. Main Execution Loop
Bắt đầu quá trình cào dữ liệu chi tiết.
> **Lưu ý:**
> * Sử dụng `tqdm` để hiển thị thanh tiến độ.
> * Script sẽ tạm nghỉ `0.5s` giữa mỗi request để tuân thủ quy tắc Rate Limiting của UQ.

In [4]:
# --- CẤU HÌNH TỐC ĐỘ ---
MAX_WORKERS = 20  # Số luồng chạy song song (Đừng để quá cao, UQ sẽ chặn. 5-10 là an toàn)

results = []
failed_courses = []

print(f"🚀 Bắt đầu cào dữ liệu với {MAX_WORKERS} luồng song song...")


# Hàm wrapper để xử lý ngoại lệ trong Thread
def process_course(code):
    try:
        # Không cần time.sleep ở đây nữa vì mạng sẽ tự tạo độ trễ tự nhiên
        # hoặc giữ time.sleep(0.1) nếu muốn cực kỳ an toàn
        return get_full_course_data(code)
    except Exception as e:
        return None

# SỬ DỤNG THREAD POOL
with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # Gửi tất cả nhiệm vụ vào Pool
    future_to_code = {executor.submit(process_course, code): code for code in course_list}
    
    # Dùng tqdm để theo dõi tiến độ khi các task hoàn thành
    for future in tqdm(concurrent.futures.as_completed(future_to_code), total=len(course_list), desc="Downloading"):
        code = future_to_code[future]
        try:
            data = future.result()
            if data:
                results.append(data)
            else:
                failed_courses.append(code)
        except Exception as exc:
            print(f"⚠️ {code} sinh ra lỗi: {exc}")
            failed_courses.append(code)


🚀 Bắt đầu cào dữ liệu với 20 luồng song song...


Downloading: 100%|██████████| 3860/3860 [11:52<00:00,  5.42it/s]


## 📊 4. Data Verification & Export
* Kiểm tra nhanh dữ liệu bằng `pandas`.
* Xuất toàn bộ dữ liệu sạch ra file `data/master_courses.json` để chuẩn bị import vào Supabase.

In [6]:
df = pd.DataFrame(results)
output_path = os.path.join('..', 'data', 'master_courses.json')
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)
    
print(f"✅ Hoàn tất! Đã cào {len(results)} môn. (Thất bại: {len(failed_courses)})")
print(f"✅ Đã lưu file thành công tại: {output_path}")

✅ Hoàn tất! Đã cào 3860 môn. (Thất bại: 0)
✅ Đã lưu file thành công tại: ../data/master_courses.json
